# Example Datasets

This is a notebook full of simple examples of common datasets one may want to download from Mortar.

- [Meter Data](#meter)
- [Temperature Data](#Temperature-Data)
    - [from Thermostats](#Thermostat-Temperature-Data)
    - [from VAVs](#VAV-Temperature-Data)

To use these examples, create a free Mortar account at [https://mortardata.org/](https://mortardata.org/) and insert your credentials below. You may also want to check out the [interactive tutorial](https://tutorial.mortardata.org/) to get a feel for the Mortar API.

In [ ]:
import pymortar
import pandas as pd

# removes extraneous output
import logging
logging.getLogger().setLevel("CRITICAL")
logging.getLogger('matplotlib').setLevel("CRITICAL")

client = pymortar.Client({
    'username': "CHANGE_THIS",  # <------------- CHANGE THIS
    'password': "CHANGE_THIS",  # <------------- CHANGE THIS
})
print("Success! Got client")

## Meter Data

This query fetches the building-level electric meters for all available sites in the Mortar testbed, downsampled to a 15-min interval. The query `meter_query` does not pull in any additional context, and the View and DataFrame definitions are minimal.

In [ ]:
meter_query = """SELECT ?meter WHERE {
    ?meter rdf:type/rdfs:subClassOf* brick:Building_Electric_Meter
};"""

meter_view = pymortar.View(
    name="meters",
    definition=meter_query,
)

meter_data = pymortar.DataFrame(
    name="meters",
    aggregation=pymortar.MEAN,
    window="15m",
    timeseries=[
        pymortar.Timeseries(
            view="meters",
            dataVars=["?meter"],
        )
    ]
)

time_range = pymortar.TimeParams(
    start="2018-01-01T00:00:00Z",
    end="2019-01-01T00:00:00Z",
)

This code accesses the Mortar API to identify the sites that have meters, and then downloads a year of data for those sites

In [ ]:
# find sites that have meters
qualified_sites = client.qualify([meter_query])
print(f'Downloading data for {len(qualified_sites.sites)} sites')

# define the request
request = pymortar.FetchRequest(
    sites = qualified_sites.sites,
    views = [meter_view],
    dataFrames = [meter_data],
    time = time_range,
)

# download the data
meter_resp = client.fetch(request)
print(f'Downloaded data: {meter_resp}')

### Downloaded Metadata

In [ ]:
meter_resp.view('meters').head()

### Downloaded Data

In [ ]:
meter_resp['meters'].head()

# Temperature Data

These views and dataframes identify and download temperature data for a set of sites. Here we demonstrate how to get temperature sensors associated with Thermostats, VAV boxes, and finally how to get *all* temperature sensor data.

All data is downloaded for the span of a year downsampled to a 15-min interval.

## Thermostat Temperature Data

In [ ]:
tstat_temp_query = """SELECT ?temp_sensor ?tstat ?zone WHERE {
    ?tstat  rdf:type  brick:Thermostat .
    ?tstat  bf:hasPoint  ?temp_sensor .
    ?temp_sensor rdf:type/rdfs:subClassOf* brick:Temperature_Sensor .
    ?tstat  bf:controls/bf:feeds+ ?zone .
    ?zone  rdf:type  brick:HVAC_Zone 
};"""

tstat_temp_view = pymortar.View(
    name = "tstat_temp",
    definition = tstat_temp_query,
)

tstat_temp_data = pymortar.DataFrame(
    name = "tstat_temp",
    aggregation = pymortar.MEAN,
    window = "15m",
    timeseries = [
        pymortar.Timeseries(
            view = "tstat_temp",
            dataVars = ["?temp_sensor"],
        )
    ]
)

time_range = pymortar.TimeParams(
    start="2018-01-01T00:00:00Z",
    end="2019-01-01T00:00:00Z",
)

In [ ]:
# find sites that have Thermostats with Temperature Sensors
qualified_sites = client.qualify([tstat_temp_query])
print(f'Downloading data for {len(qualified_sites.sites)} sites')

# define the request
request = pymortar.FetchRequest(
    sites = qualified_sites.sites,
    views = [tstat_temp_view],
    dataFrames = [tstat_temp_data],
    time = time_range,
)

# download the data
tstat_temp_resp = client.fetch(request)
print(f'Downloaded data: {tstat_temp_resp}')

### Downloaded Metadata

In [ ]:
tstat_temp_resp.view('tstat_temp').head()

### Downloaded Data

In [ ]:
tstat_temp_resp['tstat_temp'].head()

## VAV Temperature Data

You'll notice that the queries and general approach here are very similar to the Thermostat setting above. We have to change the type of the equipment in the base query in order to consider VAVs instead of Thermostats, and the relationship between the VAV and a Zone is different than the relationship between a Thermostat and a Zone.

The queries here pull in some additional context: for each temperature sensor, we get the VAV it is associated with, ~~the upstream AHU~~ and the name of the zone that the VAV is conditioning.

**Note:** this is potentially a lot of data so we are pre-emptively downsampling to a 1-hour interval. If you are using this data for any actual analysis you will probably want to reduce that to at least 15-minutes.

In [ ]:
vav_temp_query = """SELECT ?temp_sensor ?vav ?zone WHERE {
    ?vav  rdf:type/rdfs:subClassOf*  brick:VAV .
    ?vav  bf:hasPoint  ?temp_sensor .
    ?temp_sensor rdf:type/rdfs:subClassOf* brick:Temperature_Sensor .
    ?vav  bf:feeds+ ?zone .
    ?zone  rdf:type  brick:HVAC_Zone 
};"""

vav_temp_view = pymortar.View(
    name = "vav_temp",
    definition = vav_temp_query,
)

vav_temp_data = pymortar.DataFrame(
    name = "vav_temp",
    aggregation = pymortar.MEAN,
    window = "1h",
    timeseries = [
        pymortar.Timeseries(
            view = "vav_temp",
            dataVars = ["?temp_sensor"],
        )
    ]
)

time_range = pymortar.TimeParams(
    start="2018-01-01T00:00:00Z",
    end="2019-01-01T00:00:00Z",
)

In [ ]:
# find sites that have VAVs with Temperature Sensors
qualified_sites = client.qualify([vav_temp_query])
print(f'Downloading data for {len(qualified_sites.sites)} sites')

# define the request
request = pymortar.FetchRequest(
    sites = qualified_sites.sites,
    views = [vav_temp_view],
    dataFrames = [vav_temp_data],
    time = time_range,
)

# download the data
vav_temp_resp = client.fetch(request)
print(f'Downloaded data: {vav_temp_resp}')

### Downloaded Metadata

In [ ]:
vav_temp_resp.view('vav_temp').head()

### Downloaded Data

In [ ]:
vav_temp_resp['vav_temp'].head()